In [1]:
import os 
import numpy as np 
import pandas as pd
from collections import defaultdict
import itertools
import random

In [16]:
datadir = '../../datasets/goodbooks-10k-master/'

In [ ]:
#ToDo
#Filter out tags that does not make sense
#Right now I only delete the entry in the book_tags.csv if the count is less than a threshold
#We can also manuallly remove tags in the tags.csv if we think the tag does not make sense

In [17]:
#build a dict betweer goodread_book_id and the book_id used in this dataset
books = datadir + 'books.csv'
df_books = pd.read_csv(books, delimiter=',')

In [26]:
df_books.dataframeName = 'books.csv'
df_books= df_books[['book_id', 'goodreads_book_id']]
print(df_books.columns)
print(df_books.head(5))

Index([u'book_id', u'goodreads_book_id'], dtype='object')
   book_id  goodreads_book_id
0        1            2767052
1        2                  3
2        3              41865
3        4               2657
4        5               4671


In [19]:
#dict with key: goodreads_book_id, val: book_id 
book_id_dict = defaultdict(lambda:-1)
for index, row in df_books.iterrows():    
    bookId = int(row['book_id'])
    GoodReadId = int(row['goodreads_book_id'])
    book_id_dict[GoodReadId]=bookId
print(len(book_id_dict))
print(book_id_dict[1])

10000
27


In [25]:
#load book_tags file
book_tags = datadir + 'book_tags.csv'
df_book_tags = pd.read_csv(book_tags, delimiter=',')
print(len(df_book_tags))
print(df_book_tags.head(5))

999912
   goodreads_book_id  tag_id   count
0                  1   30574  167697
1                  1   11305   37174
2                  1   11557   34173
3                  1    8717   12986
4                  1   33114   12716


In [21]:
df_book_tags.dataframeName = 'book_tags.csv'
print(df_book_tags.columns)

Index([u'goodreads_book_id', u'tag_id', u'count'], dtype='object')


In [22]:
print (max(df_book_tags['goodreads_book_id']), min(df_book_tags['goodreads_book_id']))
print (max(df_book_tags['tag_id']), min(df_book_tags['tag_id']))
print (max(df_book_tags['count']), min(df_book_tags['count']))
print(len(df_book_tags))
#There are negative counts?

(33288638, 1)
(34251, 0)
(596234, -1)
999912


In [23]:
#remove entry if tag count is lower than given threshold, in which case the association between the tag and the book is
#not strong
tag_count_threshold = 500
df_book_tags = df_book_tags[df_book_tags['count'] >= tag_count_threshold]
print(len(df_book_tags))

30491


In [24]:
#convert the good_read_book_id to book_id in this dataframe
for index, row in df_book_tags.iterrows():
    bookId = book_id_dict[int(row['goodreads_book_id'])]
    df_book_tags.set_value(index,'goodreads_book_id', bookId) 
print(df_book_tags.head(5))

/home/ninad/anaconda3/envs/tfcpu-py27/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


   goodreads_book_id  tag_id   count
0                 27   30574  167697
1                 27   11305   37174
2                 27   11557   34173
3                 27    8717   12986
4                 27   33114   12716


In [27]:
# rename the column name from goodread_id to book_id
df_book_tags.rename(columns={'goodreads_book_id': 'book_id'}, inplace=True)
print(df_book_tags.head(5))

   book_id  tag_id   count
0        1   30574  167697
1        1   11305   37174
2        1   11557   34173
3        1    8717   12986
4        1   33114   12716


In [ ]:
#dict with key: book_id, val: tag_id, to list all the tags every book has
book_tag_dict = defaultdict(list)
for index, row in df_book_tags.iterrows():    
    bookId = int(row['book_id'])
    tagId = int(row['tag_id'])
    book_tag_dict[bookId].append(tagId)
print(len(book_tag_dict))

In [ ]:
#marginal counts included
#here the tag tag pair is incremented by one if a book is listed with both of the two tags
tag_tag_count =  defaultdict(lambda: 0)
for key, val in book_tag_dict.items():
    for i in range(len(val)):
        for j in range(i, len(val)):
            tag_tag_count[(val[i], val[j]) if val[i] <= val[j] else (val[j], val[i])] += 1  

In [ ]:
print(len(tag_tag_count))